<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Ojeda G.


### **Link de repositorio de GitHub:** `https://github.com/sebajedi23/MDS7202`

### Indice

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [20]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Colab Notebooks/Laboratorio de programación científica para ciencia de datos/Lab 7/'
except:
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito.
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [5]:
df_retail = pd.read_pickle(path + "online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos]

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [9]:
def custom_features(dataframe_in):
    """
    Calcula características personalizadas (RFMLP) a partir de un DataFrame de
    datos relacionados con ventas al por menor.

    Parameters
    ----------
    dataframe_in : pandas.DataFrame
        Un DataFrame que contiene información de transacciones de ventas al por
        menor, incluyendo columnas como 'Customer ID', 'InvoiceDate', 'Quantity'
        , y 'Price'.

    Returns
    -------
    result_df : pandas.DataFrame
        Un nuevo DataFrame que contiene las características calculadas de
        'Length', 'Recency', 'Frequency', 'Monetary', y 'Periodicity', para cada
         cliente identificado por 'Customer ID'.
    """
    #Creamos una copia del DataFrame para evitar cambios en el original
    df = dataframe_in.copy()

    #Convertimos los datos de la columna 'InvoiceDate' a datetime
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

    #Calculamos Length
    length_df = df.groupby('Customer ID')['InvoiceDate'].agg(lambda x: (x.max() - x.min()).days).rename('Length')

    #Calculamos Recency
    recency_df = df.groupby('Customer ID')['InvoiceDate'].max()
    recency_df = (df['InvoiceDate'].max() - recency_df).dt.days.rename('Recency')

    #Calculamos Monetary
    df['TotalPrice'] = df['Quantity'] * df['Price']
    monetary_df = df.groupby('Customer ID')['TotalPrice'].mean().rename('Monetary')

    #Calculamos Frequency y eliminamos los duplicados
    frequency_df = df.drop_duplicates(subset=['InvoiceDate', 'Customer ID']).groupby('Customer ID').size().rename('Frequency')

    #Calculamos Periodicity
    ivt_df = df.groupby('Customer ID')['InvoiceDate'].diff().dt.days
    periodicity_df = ivt_df.groupby(df['Customer ID']).std().rename('Periodicity')

    #Creamos un nuevo DataFrame que contenga las características calculadas
    result_df = pd.concat([length_df, recency_df, frequency_df, monetary_df, periodicity_df], axis=1)

    return result_df

#Guardamos el resultado en el DataFrame result
result = custom_features(df_retail).reset_index()
result.head()

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
0,12346.0,196,164,11,11.298788,21.724076
1,12347.0,37,2,2,18.638310,4.422346
2,12348.0,0,73,1,11.108000,0.000000
3,12349.0,181,42,3,26.187647,16.200990
4,12351.0,0,10,1,14.330000,0.000000


Comprobamos los valores nulos y su motivo

In [11]:
#Revisamos la presencia de valores nulos
result.isna().sum()

Customer ID      0
Length           0
Recency          0
Frequency        0
Monetary         0
Periodicity    148
dtype: int64

In [15]:
#Revisamos si alguno de los nulos tiene más de 2 visitas
result.query('Periodicity != Periodicity and Frequency > 2')

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity


In [16]:
#Revisamos los clientes con 2 visitas o menos
result.query('Periodicity != Periodicity and Frequency <= 2').sort_values('Frequency', ascending=False)

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
2446,15760.0,0,265,2,6958.17,NaN
1686,14707.0,141,70,2,69.60,NaN
3135,16692.0,32,43,2,319.35,NaN
3317,16943.0,84,174,2,241.38,NaN
1000,13776.0,0,63,2,311.99,NaN
...,...,...,...,...,...,...
1216,14079.0,0,317,1,243.63,NaN
1183,14033.0,0,190,1,109.44,NaN
1114,13942.0,0,169,1,320.00,NaN
1100,13915.0,0,29,1,35.40,NaN


Con esto notamos que los valores nulos se deben a personas que han tenido 2 o menos visitas

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [18]:
class MinMax(BaseEstimator, TransformerMixin):

    def fit(self, X):
        self.min_vals = X.min()
        self.max_vals = X.max()
        return self

    def transform(self, X):
        X_transformed = (X - self.min_vals) / (self.max_vals - self.min_vals)

        return X_transformed

### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo.

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior.
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [41]:
#Convertimos en transformador la función de calculo de características
feature_extractor = FunctionTransformer(custom_features, validate=False)

#Cambiamos los posibles NaN por 0
imputer = SimpleImputer(strategy='constant', fill_value=0.0)

#Escalamos las columnas RFMLP
transformer = ColumnTransformer(
    transformers=[
        ('minmax',
         MinMax(),
         slice(0, None))
    ],
    remainder='passthrough'
)

#Reducimos la dimensionalidad a 2 componentes principales
tsne = TSNE(n_components=2)

#Creamos el pipeline para los pasos solicitados
pipeline = Pipeline([
    ('feature_extraction', feature_extractor),
    ('imputation', imputer),
    ('scaling', transformer),
    ('dimension_reduction', tsne)
])

#Aplicamos el pipeline al DataFrame entregado
reduced_features = pipeline.fit_transform(df_retail)

#Creamos un DataFrame con la dimensionalidad reducida
reduced_df = pd.DataFrame(data=reduced_features, columns=["Component 1", "Component 2"])

fig = px.scatter(
    reduced_df, x="Component 1", y="Component 2",
    title="Reducción de dimensionalidad t-SNE con 2 componentes principales",
    labels={'Component 1': 'Componente principal 1', 'Component 2': 'Componente principal 2'},
    width=800, height=600
)

fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [43]:
#Definimos la cantidad de clusters k a probar entre 1 y 20
k_values = range(1, 21)
inertia_values = []

#Creamos un nuevo pipeline para cada numero de clusters
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_pipeline = Pipeline([
        ('features', feature_extractor),
        ('impute', imputer),
        ('scaler', transformer),
        ('clustering', kmeans)
    ])

    #Aplicamos el pipeline y guardamos la inercia para el valor de k
    cluster_pipeline.fit_transform(df_retail)
    inertia = cluster_pipeline.named_steps['clustering'].inertia_
    inertia_values.append(inertia)

#Creamos un DataFrame con los valores de k y la inercia
elbow_data = pd.DataFrame({'k': k_values, 'Inertia': inertia_values})

#Graficamos lo obtenido
fig = px.line(elbow_data, x='k', y='Inertia', title='Método del Codo para Determinar el Valor Óptimo de k')
fig.update_traces(mode='lines+markers')
fig.update_xaxes(title='Número de Clusters (k)')
fig.update_yaxes(title='Inertia')
fig.show()

A partir de lo observado se concluye que los mejores valores están entre k=3 y k=5, ya que la derivada de la inercia respecto a k es relativamente similar entre estos valores y después de ellos comienza a disminuir, generándo menos beneficios con la inclusión de clusters adicionales. Por lo tanto al pasar de 3 a 4 clusters se obtiene prácticamente los mismos beneficios que al pasar de 4 a 5.

Por lo anterior, se elige el valor de k=4, ya que al ser el valor de en medio presenta menos riesgo de caer en una subsegmentación que k=3, a la vez que menos riesgo de caer en una sobresegmentación que k=5, aunque probablemente estos valores serían igual de aceptables.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [44]:
#Definimos el valor optimo de k
optimal_k = 4

#Definimos el KMeans con el k optimo
kmeans = KMeans(n_clusters=optimal_k, random_state=42)

#Creamos el pipeline para este KMeans
cluster_pipeline = Pipeline([
    ('features', feature_extractor),
    ('impute', imputer),
    ('scaler', transformer),
    ('clustering', kmeans)
])

#Aplicamos la transformacion
cluster_pipeline.fit_transform(df_retail)

#Obtenemos las etiquetas para cada cluster
labels = cluster_pipeline.named_steps['clustering'].labels_

#Copiamos el DataFrame e imputamos posibles NaN
df_retail_2 = custom_features(df_retail)
Segmentation_df = imputer.fit_transform(df_retail_2)

#Creamos un DataFrame con lo anterior
Segmentation_df = pd.DataFrame(Segmentation_df, columns=df_retail_2.columns)

#Creamos una columna 'Cluster' al DataFrame que contenga las etiquetas de los datos
Segmentation_df['Cluster'] = labels


#Calculamos los promedios para cada atributo agrupados por clúster
cluster_means = Segmentation_df.groupby('Cluster').agg({
    'Length': 'mean',
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'Periodicity': 'mean',
    'Cluster': 'count'  # Agregar una columna de recuento de datos por clúster
})

# Renombrar la columna de recuento
cluster_means = cluster_means.rename(columns={'Cluster': ''})


# Mostrar la tabla de promedios
cluster_means

,Length,Recency,Frequency,Monetary,Periodicity,
Cluster,,,,,,
0,279.260571,35.906857,8.134286,32.487854,20.874258,1750
1,41.589919,54.037959,2.126945,26.979737,5.720115,1607
2,0.000000,263.000000,1.500000,8955.835000,0.000000,2
3,23.309948,250.491099,1.594764,43.392481,3.575969,955


Adicionalmente podemos revisar otras métricas para cada cluster

In [45]:
Segmentation_df.describe()

,Length,Recency,Frequency,Monetary,Periodicity,Cluster
count,4314.000000,4314.000000,4314.000000,4314.000000,4314.000000,4314.000000
mean,133.936486,90.269124,4.445758,36.986943,11.390177,1.037552
std,132.827714,96.943482,8.112355,217.659847,16.083429,1.136001
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,17.000000,1.000000,11.185231,0.000000,0.000000
50%,105.000000,52.000000,2.000000,17.357104,6.974007,1.000000
75%,253.750000,135.000000,5.000000,24.813257,16.547550,1.000000
max,373.000000,373.000000,204.000000,10953.500000,181.169120,3.000000


Cluster 0: Este cluster podría representar a los clientes relativamente longevos, con recencia baja y una frecuencia moderada de transacciones. Estos clientes no gastan grandes cantidades de dinero en cada transacción, pero han sido relativamente regulares en su participación por un tiempo considerable.

Cluster 1: Este cluster representa a clientes menos longevos que el cluster 0, pero con recencia más alta y una frecuencia menor de transacciones. También tienen un gasto por transacción moderado.

Cluster 2: Este cluster es interesante, ya que parece representar a clientes nuevos que han realizado transacciones por valores monetarios muy altos. Podrían ser compradores ocasionales o compradores únicos enfocados en productos especializados.

Cluster 3: Este cluster incluye clientes relativamente nuevos pero que realizan compras por valores medios o altos.

Con lo anterior se podría decir que el retail posee variados tipos de clientes con diferentes patrones de comportamiento. Por ejemplo, los clientes en el Cluster 2 podrían ser considerados "grandes compradores únicos", mientras que los del Cluster 1 podrían ser "compradores ocasionales", el Cluster 0 podría ser "clientes fidelizados" debido a su alta longevidad y frecuencia, y el Cluster 3 podría representar "clientes de interés", ya que son relativamente nuevos pero debido a los montos de sus compras puede ser importante retenerlos.

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [47]:
#Creamos un nuevo DataFrame con los datos
TSNE_df = pd.DataFrame(data=reduced_df, columns=['Component 1', 'Component 2'])
TSNE_df['Cluster'] = labels

# Crear el gráfico de dispersión con Plotly
fig = px.scatter(TSNE_df, x='Component 1', y='Component 2', color='Cluster', title='Segmentación de clientes usando KMeans', opacity=0.5, color_continuous_scale='viridis')
fig.update_layout(width=800, height=700)
fig.update_xaxes(title_text='Componente principal 1')
fig.update_yaxes(title_text='Componente principal 2')
fig.show()

En el gráfico se puede observar claramente la distribución y separación de los clusters, siendo quizá el más complejo de ver el cluster 2, aunque esto se debe a su baja representatividad, teniendo sólo 2 clientes en él correspondientes a 2 puntos cerca de las coordenadas (18,-16) en el gráfico (se probó bajar el número de clusters a 3 pero este resultado no cambia, por lo que se intuye que este cluster corresponde a un tipo de cliente muy específico y poco común).

Con esto se puede concluir que los clusters se separan bien, aunque sería interesante tener más clientes en el cluster 2 para corroborar si este se solapa con algún otro o si está bien definido al igual que los demás.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>